## Creación de una base de datos unificada.

![Esquema de la base de datos](../img/database_schema.png)

In [17]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd
import logging

Creo el motor de base de datos.

In [21]:
engine = create_engine('sqlite:///../database.db', echo=False)

In [20]:
def crear_tabla(nombre, data: pd.DataFrame):
    try:
        data.to_sql(name=nombre, con=engine, index=False, if_exists='replace')
        print(f'Tabla "{nombre}" creada con exito.')
    except:
        raise "Error al crear la tabla."
    
def obtener_tabla(nombre, engine):
    try:
        return pd.read_sql_table(table_name=nombre, con=engine)
    except Exception as e:
        raise ValueError("❌ No se encuentra la tabla en la base de datos.") from e


# Tablas de hecho.

### Tabla `clima`.

In [64]:
df_clima = pd.read_parquet('../data/datos_climaticos.parquet')

In [65]:
df_clima = df_clima.loc[df_clima.fecha > "2017-12-31"]\
            .sort_values(by=['id_estacion', 'fecha'])\
            .reset_index(drop=True)
            

In [66]:
crear_tabla('clima', df_clima)

2025-10-24 20:22:42,234 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-24 20:22:42,239 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clima")
2025-10-24 20:22:42,239 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:42,240 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clima")
2025-10-24 20:22:42,241 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:42,242 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:42,243 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:42,244 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:42,244 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:42,245 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("clima")
2025-10-24 20:22:42,246 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:4

### Tabla `contagios`

In [4]:
df_contagios = pd.read_csv('../data/dengue-final.csv')
df_contagios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68126 entries, 0 to 68125
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_uta                   68126 non-null  int64  
 1   departamento_nombre      68126 non-null  object 
 2   provincia_nombre         68126 non-null  object 
 3   ano                      68126 non-null  int64  
 4   semanas_epidemiologicas  68126 non-null  float64
 5   grupo_edad_id            68126 non-null  int64  
 6   grupo_edad_desc          68126 non-null  object 
 7   cantidad_casos           68126 non-null  float64
 8   poblacion                68126 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 4.7+ MB


In [5]:
df_contagios.head()

,id_uta,departamento_nombre,provincia_nombre,ano,semanas_epidemiologicas,grupo_edad_id,grupo_edad_desc,cantidad_casos,poblacion
0,2003,comuna 3,ciudad de buenos aires,2018,15.0,6,de 15 a 19 anos,1.0,192945
1,2004,comuna 4,ciudad de buenos aires,2018,6.0,6,de 15 a 19 anos,1.0,239279
2,6091,berazategui,buenos aires,2018,11.0,7,de 20 a 24 anos,1.0,358262
3,6091,berazategui,buenos aires,2018,14.0,9,de 35 a 44 anos,2.0,358262
4,6091,berazategui,buenos aires,2018,19.0,11,mayores de 65 anos,1.0,358262


In [6]:
df_contagios = df_contagios.rename(columns=
                    {
                        'departamento_nombre': 'departamento',
                        'provincia_nombre': 'provincia'
                    })

In [8]:
df_contagios.provincia.unique()

array(['ciudad de buenos aires', 'buenos aires', 'mendoza', 'santa fe',
       'rio negro', 'cordoba', 'corrientes', 'formosa', 'chaco',
       'la rioja', 'san luis', 'entre rios', 'misiones', 'salta',
       'santiago del estero', 'tucuman', 'jujuy', 'catamarca', 'neuquen',
       'chubut', 'la pampa', 'san juan', 'santa cruz',
       'tierra del fuego antartida e islas del atlantico sur'],
      dtype=object)

In [9]:
PROV_REEMPLAZO = {
    "ciudad de buenos aires": "BUENOS AIRES",
    "buenos aires": "BUENOS AIRES",
    "mendoza": "MENDOZA",
    "santa fe": "SANTA FE",
    "rio negro": "RÍO NEGRO",
    "cordoba": "CÓRDOBA",
    "corrientes": "CORRIENTES",
    "formosa": "FORMOSA",
    "chaco": "CHACO",
    "la rioja": "LA RIOJA",
    "san luis": "SAN LUIS",
    "entre rios": "ENTRE RÍOS",
    "misiones": "MISIONES",
    "salta": "SALTA",
    "santiago del estero": "SANTIAGO DEL ESTERO",
    "tucuman": "TUCUMÁN",
    "jujuy": "JUJUY",
    "catamarca": "CATAMARCA",
    "neuquen": "NEUQUÉN",
    "chubut": "CHUBUT",
    "la pampa": "LA PAMPA",
    "san juan": "SAN JUAN",
    "santa cruz": "SANTA CRUZ",
    "tierra del fuego antartida e islas del atlantico sur": "TIERRA DEL FUEGO"
}

df_contagios.provincia = df_contagios.provincia.map(PROV_REEMPLAZO) 

In [13]:
cols_2upper = ['departamento', 'grupo_edad_desc']
for col in cols_2upper:
    df_contagios[col] = df_contagios[col].str.upper()

df_contagios.sample(3)

,id_uta,departamento,provincia,ano,semanas_epidemiologicas,grupo_edad_id,grupo_edad_desc,cantidad_casos,poblacion
13226,82021,CASTELLANOS,SANTA FE,2022,19.0,6,DE 15 A 19 ANOS,1.0,211047
19887,66084,LA CANDELARIA,SALTA,2023,16.0,7,DE 20 A 24 ANOS,1.0,6628
18161,10049,CATAMARCA CAPITAL,CATAMARCA,2023,14.0,5,DE 10 A 14 ANOS,1.0,188824


In [23]:
tabla_provincias = obtener_tabla('provincias', engine)
tabla_provincias.head()

,provincia,id_provincia
0,BUENOS AIRES,0
1,CABA,1
2,CATAMARCA,2
3,CHACO,3
4,CHUBUT,4


In [26]:
df_contagios = df_contagios.merge(tabla_provincias, on='provincia')

In [27]:
crear_tabla('contagios', df_contagios)

Tabla "contagios" creada con exito.


# Tablas de dimensiones.

### Tabla `calendario`.

In [70]:
df_calendario = pd.read_csv('../data/calendario.csv')
df_calendario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24004 entries, 0 to 24003
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fecha            24004 non-null  object
 1   anio             24004 non-null  int64 
 2   mes              24004 non-null  int64 
 3   dia              24004 non-null  int64 
 4   trimestre        24004 non-null  int64 
 5   semestre         24004 non-null  int64 
 6   quincena         24004 non-null  int64 
 7   semanaMes        24004 non-null  int64 
 8   semana           24004 non-null  int64 
 9   diaSemana        24004 non-null  object
 10  diaNumeroSemana  24004 non-null  int64 
 11  bisiesto         24004 non-null  bool  
dtypes: bool(1), int64(9), object(2)
memory usage: 2.0+ MB


In [71]:
df_calendario

,fecha,anio,mes,dia,trimestre,semestre,quincena,semanaMes,semana,diaSemana,diaNumeroSemana,bisiesto
0,1960-01-01,1960,1,1,1,1,1,1,53,Viernes,4,True
1,1960-01-02,1960,1,2,1,1,1,1,53,Sábado,5,True
2,1960-01-03,1960,1,3,1,1,1,1,53,Domingo,6,True
3,1960-01-04,1960,1,4,1,1,1,1,1,Lunes,0,True
4,1960-01-05,1960,1,5,1,1,1,1,1,Martes,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...
23999,2025-09-15,2025,9,15,3,2,1,3,38,Lunes,0,False
24000,2025-09-16,2025,9,16,3,2,2,3,38,Martes,1,False
24001,2025-09-17,2025,9,17,3,2,2,3,38,Miércoles,2,False
24002,2025-09-18,2025,9,18,3,2,2,3,38,Jueves,3,False


In [72]:
crear_tabla('calendario', df_calendario)

2025-10-24 20:22:44,187 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-24 20:22:44,191 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("calendario")
2025-10-24 20:22:44,192 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,193 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("calendario")
2025-10-24 20:22:44,193 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,194 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:44,194 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,195 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:44,195 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,196 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("calendario")
2025-10-24 20:22:44,196 INFO sqlalchemy.engine.Engine [raw sql] ()
202

### Tabla `ubicacion`

In [73]:
df_estaciones = pd.read_csv('../data/estaciones-meteorologicas-inta.csv')
df_departamentos = pd.read_csv('../data/departamentos_con_estacion.csv')

In [74]:
lista_provincias = df_contagios.provincia_nombre.apply(lambda x: ' '.join(p.capitalize() for p in x.split(' '))).unique()
print(lista_provincias)

['Ciudad De Buenos Aires' 'Buenos Aires' 'Mendoza' 'Santa Fe' 'Rio Negro'
 'Cordoba' 'Corrientes' 'Formosa' 'Chaco' 'La Rioja' 'San Luis'
 'Entre Rios' 'Misiones' 'Salta' 'Santiago Del Estero' 'Tucuman' 'Jujuy'
 'Catamarca' 'Neuquen' 'Chubut' 'La Pampa' 'San Juan' 'Santa Cruz'
 'Tierra Del Fuego Antartida E Islas Del Atlantico Sur']


In [75]:
df_provincias = pd.DataFrame({'provincia': lista_provincias})
df_provincias

,provincia
0,Ciudad De Buenos Aires
1,Buenos Aires
2,Mendoza
3,Santa Fe
4,Rio Negro
5,Cordoba
6,Corrientes
7,Formosa
8,Chaco
9,La Rioja


In [76]:
mapeo_renombrar = {
    'Cordoba': 'Córdoba', 
    'Tierra Del Fuego Antartida E Islas Del Atlantico Sur': 'Tierra del Fuego',
    'Ciudad De Buenos Aires': 'CABA',
    'Tucuman': 'Tucumán',
    'Neuquen': 'Neuquén',
    'Entre Rios': 'Entre Ríos',
    'Rio Negro': 'Río Negro'
}

df_provincias = df_provincias.replace(mapeo_renombrar).sort_values('provincia').reset_index(drop=True)
df_provincias

,provincia
0,Buenos Aires
1,CABA
2,Catamarca
3,Chaco
4,Chubut
5,Corrientes
6,Córdoba
7,Entre Ríos
8,Formosa
9,Jujuy


In [77]:
df_provincias['id_provincia'] = range(len(df_provincias))
df_provincias

,provincia,id_provincia
0,Buenos Aires,0
1,CABA,1
2,Catamarca,2
3,Chaco,3
4,Chubut,4
5,Corrientes,5
6,Córdoba,6
7,Entre Ríos,7
8,Formosa,8
9,Jujuy,9


In [78]:
df_provincias.provincia = df_provincias.provincia.str.upper()
df_provincias.head()

,provincia,id_provincia
0,BUENOS AIRES,0
1,CABA,1
2,CATAMARCA,2
3,CHACO,3
4,CHUBUT,4


In [79]:
crear_tabla('provincias', df_provincias)

2025-10-24 20:22:44,534 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-24 20:22:44,536 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("provincias")
2025-10-24 20:22:44,538 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,539 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("provincias")
2025-10-24 20:22:44,540 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,541 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:44,541 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,542 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:44,542 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,543 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("provincias")
2025-10-24 20:22:44,543 INFO sqlalchemy.engine.Engine [raw sql] ()
202

### Tabla `localidades`

In [80]:
df_estaciones.head()

,Id,Nombre,Tipo,Localidad,Provincia,Latitud,Longitud,Altura,Id Interno,Ubicacion,Desde,Hasta
0,413,25 de Mayo - EEA Pergamino,Nimbus THP,25 de Mayo,Buenos Aires,-35.48,-60.13,85,A872872,CC 18 CP6660,04/09/2012,13/09/2025
1,682,5000,Nimbus THP,Hurlingham,Buenos Aires,-85.00,-55.00,0,A875000,-,20/11/2019,13/09/2025
2,683,5001,Nimbus THP,Hurlingham,Buenos Aires,-85.00,-60.00,0,A875001,-,20/11/2019,13/09/2025
3,704,993 - LAB,Nimbus THP,loc_prueba,Sin asignar,-84.00,-70.00,0,A872993,LABORATORIO,08/03/2022,26/08/2024
4,705,994 - LAB,Nimbus THP,loc_prueba,Sin asignar,-84.00,-65.00,0,A872994,LABORATORIO,01/01/2012,28/08/2025


In [81]:
df_departamentos.head()

,departamento_id,departamento_nombre,lat_dep,lon_dep,estacion_id,estacion_id_interno,estacion_nombre,lat_est,lon_est,distancia_km
0,82105,san jeronimo,-32.239935,-61.231266,297,A872868,Las Rosas - EEA Oliveros,-32.49,-61.57,42.253384
1,38112,yavi,-22.329812,-65.825982,516,A872923,Abra Pampa - EEA Abra Pampa,-22.80,-65.83,52.069337
2,62035,el cuy,-39.866718,-68.703438,519,A872947,Plottier - IPAF Patagonia,-38.95,-68.33,106.737771
3,6623,pergamino,-33.589305,-60.772765,236,A872814,Alfonso - EEA Pergamino,-33.91,-60.84,36.112224
4,6868,villa gesell,-37.336673,-57.032501,368,A872833,Las Armas - EEA Cuenca Salado,-37.09,-57.87,79.218609


In [82]:
df_departamentos.departamento_nombre = df_departamentos.departamento_nombre.apply(lambda x: ' '.join(list(p.capitalize() for p in x.split(' '))))

In [83]:
merged = pd.merge(
    df_departamentos,
    df_estaciones,
    left_on='estacion_id_interno',
    right_on='Id Interno')

In [84]:
merged.isna().sum()

departamento_id         0
departamento_nombre     0
lat_dep                 0
lon_dep                 0
estacion_id             0
estacion_id_interno     0
estacion_nombre         0
lat_est                 0
lon_est                 0
distancia_km            0
Id                      0
Nombre                  0
Tipo                    0
Localidad               0
Provincia               0
Latitud                 0
Longitud                0
Altura                  0
Id Interno              0
Ubicacion              27
Desde                   0
Hasta                   0
dtype: int64

In [85]:
merged_mini = merged[[
    'departamento_id',
    'departamento_nombre',
    'Localidad'
]]

In [86]:
len(merged_mini.departamento_nombre.unique())

458

In [87]:
len(merged_mini.Localidad.unique())


142

In [88]:
df_localidades = df_estaciones[['Localidad', 'Provincia', 'Latitud', 'Longitud']]
df_localidades

,Localidad,Provincia,Latitud,Longitud
0,25 de Mayo,Buenos Aires,-35.48,-60.13
1,Hurlingham,Buenos Aires,-85.00,-55.00
2,Hurlingham,Buenos Aires,-85.00,-60.00
3,loc_prueba,Sin asignar,-84.00,-70.00
4,loc_prueba,Sin asignar,-84.00,-65.00
...,...,...,...,...
164,Manfredi,Córdoba,-31.94,-65.22
165,Villa Mercedes,San Luis,-33.66,-65.41
166,Villa Paranacito,Entre Rios,-33.71,-58.65
167,Villa Ramallo,Buenos Aires,-33.52,-60.11


In [89]:
# columnas a minusculas
df_localidades.columns = list(col.lower().strip() for col in df_localidades.columns)

In [90]:
# eliminado de duplicados
df_localidades = df_localidades.drop_duplicates(subset=['localidad', 'provincia'])

In [91]:
# eliminado de filas
df_localidades = df_localidades.drop(df_localidades.loc[df_localidades.provincia == 'Sin asignar'].index)

In [92]:
# ordeno y asignos ids
df_localidades = df_localidades.sort_values(by=['provincia', 'localidad']).reset_index(drop=True)

In [93]:
df_localidades.localidad = df_localidades.localidad.str.upper()
df_localidades.provincia = df_localidades.provincia.str.upper()

In [94]:
df_localidades['id_localidad'] = range(len(df_localidades))
df_localidades

,localidad,provincia,latitud,longitud,id_localidad
0,25 DE MAYO,BUENOS AIRES,-35.48,-60.13,0
1,ARRECIFES,BUENOS AIRES,-34.05,-60.14,1
2,BALCARCE,BUENOS AIRES,-37.76,-58.30,2
3,BORDENAVE,BUENOS AIRES,-37.75,-63.08,3
4,CHASCOMUS,BUENOS AIRES,-35.74,-58.05,4
...,...,...,...,...,...
140,SACHAYOJ,SANTIAGO DEL ESTERO,-26.46,-61.81,140
141,SILIPICA,SANTIAGO DEL ESTERO,-28.02,-64.23,141
142,VACA HUAÑUNA,SANTIAGO DEL ESTERO,-27.47,-63.47,142
143,SAN SEBASTIAN,TIERRA DEL FUEGO,-52.89,-68.45,143


In [95]:
PROV_MAP = {
    "BUENOS AIRES": "BUENOS AIRES",
    "CATAMARCA": "CATAMARCA",
    "CHACO": "CHACO",
    "CHUBUT": "CHUBUT",
    "CORRIENTES": "CORRIENTES",
    "CÓRDOBA": "CÓRDOBA",
    "ENTRE RIOS": "ENTRE RÍOS",
    "FORMOSA": "FORMOSA",
    "JUJUY": "JUJUY",
    "LA PAMPA": "LA PAMPA",
    "LA RIOJA": "LA RIOJA",
    "MENDOZA": "MENDOZA",
    "MISIONES": "MISIONES",
    "NEUQUEN": "NEUQUÉN",
    "RÍO NEGRO": "RÍO NEGRO",
    "SALTA": "SALTA",
    "SAN JUAN": "SAN JUAN",
    "SAN LUIS": "SAN LUIS",
    "SANTA CRUZ": "SANTA CRUZ",
    "SANTA FE": "SANTA FE",
    "SANTIAGO DEL ESTERO": "SANTIAGO DEL ESTERO",
    "TIERRA DEL FUEGO": "TIERRA DEL FUEGO",
    "TUCUMAN": "TUCUMÁN"
}


df_localidades.provincia = df_localidades.provincia.map(PROV_MAP)


In [96]:
df_localidades.head()

,localidad,provincia,latitud,longitud,id_localidad
0,25 DE MAYO,BUENOS AIRES,-35.48,-60.13,0
1,ARRECIFES,BUENOS AIRES,-34.05,-60.14,1
2,BALCARCE,BUENOS AIRES,-37.76,-58.30,2
3,BORDENAVE,BUENOS AIRES,-37.75,-63.08,3
4,CHASCOMUS,BUENOS AIRES,-35.74,-58.05,4


In [97]:
df_localidades = df_localidades.merge(df_provincias, on='provincia')
df_localidades

,localidad,provincia,latitud,longitud,id_localidad,id_provincia
0,25 DE MAYO,BUENOS AIRES,-35.48,-60.13,0,0
1,ARRECIFES,BUENOS AIRES,-34.05,-60.14,1,0
2,BALCARCE,BUENOS AIRES,-37.76,-58.30,2,0
3,BORDENAVE,BUENOS AIRES,-37.75,-63.08,3,0
4,CHASCOMUS,BUENOS AIRES,-35.74,-58.05,4,0
...,...,...,...,...,...,...
140,SACHAYOJ,SANTIAGO DEL ESTERO,-26.46,-61.81,140,21
141,SILIPICA,SANTIAGO DEL ESTERO,-28.02,-64.23,141,21
142,VACA HUAÑUNA,SANTIAGO DEL ESTERO,-27.47,-63.47,142,21
143,SAN SEBASTIAN,TIERRA DEL FUEGO,-52.89,-68.45,143,22


In [98]:
crear_tabla('localidades', df_localidades)

2025-10-24 20:22:44,879 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-24 20:22:44,881 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("localidades")
2025-10-24 20:22:44,882 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,883 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("localidades")
2025-10-24 20:22:44,883 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,884 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:44,885 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,886 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:44,886 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:44,887 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("localidades")
2025-10-24 20:22:44,887 INFO sqlalchemy.engine.Engine [raw sql] ()


### Tabla `departamentos`

In [99]:
df_estaciones.head()

,Id,Nombre,Tipo,Localidad,Provincia,Latitud,Longitud,Altura,Id Interno,Ubicacion,Desde,Hasta
0,413,25 de Mayo - EEA Pergamino,Nimbus THP,25 de Mayo,Buenos Aires,-35.48,-60.13,85,A872872,CC 18 CP6660,04/09/2012,13/09/2025
1,682,5000,Nimbus THP,Hurlingham,Buenos Aires,-85.00,-55.00,0,A875000,-,20/11/2019,13/09/2025
2,683,5001,Nimbus THP,Hurlingham,Buenos Aires,-85.00,-60.00,0,A875001,-,20/11/2019,13/09/2025
3,704,993 - LAB,Nimbus THP,loc_prueba,Sin asignar,-84.00,-70.00,0,A872993,LABORATORIO,08/03/2022,26/08/2024
4,705,994 - LAB,Nimbus THP,loc_prueba,Sin asignar,-84.00,-65.00,0,A872994,LABORATORIO,01/01/2012,28/08/2025


In [100]:
departamentos = df_departamentos.merge(
    df_estaciones[['Id Interno', 'Localidad', 'Provincia']], 
    left_on='estacion_id_interno',
    right_on='Id Interno')

In [101]:
departamentos = departamentos[[
    'departamento_id',
    'departamento_nombre',
    'lat_dep',
    'lon_dep',
    'Localidad',
    'Provincia'
]]

departamentos = departamentos.rename(columns={'Provincia': 'provincia', 'Localidad': 'localidad'})


In [102]:
departamentos.departamento_nombre = departamentos.departamento_nombre.str.upper().str.strip()
departamentos.localidad = departamentos.localidad.str.upper().str.strip()
departamentos.provincia = departamentos.provincia.str.upper().str.strip()

departamentos.head()

,departamento_id,departamento_nombre,lat_dep,lon_dep,localidad,provincia
0,82105,SAN JERONIMO,-32.239935,-61.231266,LAS ROSAS,SANTA FE
1,38112,YAVI,-22.329812,-65.825982,ABRA PAMPA,JUJUY
2,62035,EL CUY,-39.866718,-68.703438,PLOTTIER,NEUQUEN
3,6623,PERGAMINO,-33.589305,-60.772765,MARIANO ALFONSO,BUENOS AIRES
4,6868,VILLA GESELL,-37.336673,-57.032501,LAS ARMAS,BUENOS AIRES


In [103]:
departamentos.provincia = departamentos.provincia.map(PROV_MAP)

In [104]:
departamentos = departamentos.merge(df_provincias, on='provincia').merge(df_localidades[['id_localidad', 'localidad']], on='localidad')
departamentos

,departamento_id,departamento_nombre,lat_dep,lon_dep,localidad,provincia,id_provincia,id_localidad
0,82105,SAN JERONIMO,-32.239935,-61.231266,LAS ROSAS,SANTA FE,20,128
1,38112,YAVI,-22.329812,-65.825982,ABRA PAMPA,JUJUY,9,78
2,62035,EL CUY,-39.866718,-68.703438,PLOTTIER,NEUQUÉN,14,102
3,6623,PERGAMINO,-33.589305,-60.772765,MARIANO ALFONSO,BUENOS AIRES,0,14
4,6868,VILLA GESELL,-37.336673,-57.032501,LAS ARMAS,BUENOS AIRES,0,11
...,...,...,...,...,...,...,...,...
506,62007,ADOLFO ALSINA,-40.633583,-63.509506,VIEDMA,RÍO NEGRO,15,109
507,62028,CONESA,-40.220566,-64.070602,GUARDIA MITRE,RÍO NEGRO,15,106
508,6553,MONTE HERMOSO,-38.995358,-61.215202,CORONEL DORREGO,BUENOS AIRES,0,5
509,22084,LIBERTADOR GENERAL SAN MARTIN,-26.514810,-59.225561,PIRANE,FORMOSA,8,77


In [105]:
departamentos = departamentos.rename(
    columns={'lat_dep': 'latitud', 'lon_dep': 'longitud', 'departamento_id': 'id_departamento', 'departamento_nombre': 'departamento'})\
    .drop(['localidad', 'provincia'], axis=1)
departamentos.sample(2)

,id_departamento,departamento,latitud,longitud,id_provincia,id_localidad
228,18168,SANTO TOME,-28.518782,-56.055443,5,60
150,74028,CHACABUCO,-32.563398,-65.031531,18,114


In [106]:
# finalmente ordeno ascendentemente por el nombre del departamento

departamentos.sort_values(by='departamento')

,id_departamento,departamento,latitud,longitud,id_provincia,id_localidad
278,22126,1 DE MAYO,-27.228550,-58.904051,3,34
125,22036,12 DE OCTUBRE,-27.345879,-61.434143,3,32
139,22039,2 DE ABRIL,-27.597486,-61.131971,3,37
35,6854,25 DE MAYO,-35.281385,-60.261818,0,0
162,70126,25 DE MAYO,-31.730632,-68.345013,17,112
...,...,...,...,...,...,...
1,38112,YAVI,-22.329812,-65.825982,9,78
436,90119,YERBA BUENA,-26.778964,-65.285321,23,144
393,58112,ZAPALA,-38.822468,-70.008003,14,102
17,6882,ZARATE,-33.980419,-59.284686,0,12


In [107]:
crear_tabla('departamentos', departamentos)

2025-10-24 20:22:45,097 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-24 20:22:45,100 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("departamentos")
2025-10-24 20:22:45,101 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:45,102 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("departamentos")
2025-10-24 20:22:45,103 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:45,103 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:45,104 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:45,105 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 20:22:45,105 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:45,106 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("departamentos")
2025-10-24 20:22:45,106 INFO sqlalchemy.engine.Engine [raw sq

### Tabla `estaciones`

In [108]:
df_estaciones = df_estaciones.drop(['Altura', 'Ubicacion', 'Desde', 'Hasta', 'Tipo'], axis=1)

In [109]:
depa = df_departamentos[['estacion_nombre', 'estacion_id_interno', 'estacion_id']]

In [110]:
estaciones = pd.merge(
    df_estaciones,
    depa,
    left_on='Id Interno',
    right_on='estacion_id_interno'
)

estaciones

,Id,Nombre,Localidad,Provincia,Latitud,Longitud,Id Interno,estacion_nombre,estacion_id_interno,estacion_id
0,413,25 de Mayo - EEA Pergamino,25 de Mayo,Buenos Aires,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
1,413,25 de Mayo - EEA Pergamino,25 de Mayo,Buenos Aires,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
2,413,25 de Mayo - EEA Pergamino,25 de Mayo,Buenos Aires,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
3,413,25 de Mayo - EEA Pergamino,25 de Mayo,Buenos Aires,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
4,413,25 de Mayo - EEA Pergamino,25 de Mayo,Buenos Aires,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
...,...,...,...,...,...,...,...,...,...,...
525,694,Villa Paranacito-EEA C. del Ur,Villa Paranacito,Entre Rios,-33.71,-58.65,A872804,Villa Paranacito-EEA C. del Ur,A872804,694
526,694,Villa Paranacito-EEA C. del Ur,Villa Paranacito,Entre Rios,-33.71,-58.65,A872804,Villa Paranacito-EEA C. del Ur,A872804,694
527,695,Villa Ramallo - EEA San Pedro,Villa Ramallo,Buenos Aires,-33.52,-60.11,A872948,Villa Ramallo - EEA San Pedro,A872948,695
528,695,Villa Ramallo - EEA San Pedro,Villa Ramallo,Buenos Aires,-33.52,-60.11,A872948,Villa Ramallo - EEA San Pedro,A872948,695


In [111]:
estaciones = estaciones.drop(estaciones.loc[estaciones.Provincia == 'Sin asignar'].index)
estaciones.sample(2)

,Id,Nombre,Localidad,Provincia,Latitud,Longitud,Id Interno,estacion_nombre,estacion_id_interno,estacion_id
281,454,Junín - EEA Junin Mendoza,Junin,Mendoza,-33.12,-68.48,A872938,Junín - EEA Junin Mendoza,A872938,454
82,427,Caa Cati - EEA Corrientes,Caá Catí,Corrientes,-27.75,-57.62,A872921,Caa Cati - EEA Corrientes,A872921,427


In [112]:
# columnas a minusculas

estaciones.columns = list(col.lower().strip() for col in estaciones.columns)

In [113]:
estaciones.nombre = estaciones.nombre.str.upper()
estaciones.localidad = estaciones.localidad.str.upper()
estaciones.provincia = estaciones.provincia.str.upper()

estaciones.sample(5)

,id,nombre,localidad,provincia,latitud,longitud,id interno,estacion_nombre,estacion_id_interno,estacion_id
360,496,MONTE QUEMADO - EEA QUIMILÍ,MONTE QUEMADO,SANTIAGO DEL ESTERO,-25.86,-62.71,A872954,Monte Quemado - EEA Quimilí,A872954,496
439,291,SAN JORGE - EEA RAFAELA,SAN JORGE,SANTA FE,-31.95,-61.83,A872863,San Jorge - EEA Rafaela,A872863,291
72,491,BELLA VISTA - EEA BELLA VISTA,BELLA VISTA,CORRIENTES,-28.45,-58.99,A872951,Bella Vista - EEA Bella Vista,A872951,491
103,79,CASTELAR - ICYA CIRN,HURLINGHAM,BUENOS AIRES,-34.61,-58.67,A872801,Castelar - ICyA CIRN,A872801,79
77,518,BORDENAVE - EEA BORDENAVE,BORDENAVE,BUENOS AIRES,-37.75,-63.08,A872817,Bordenave - EEA Bordenave,A872817,518


In [114]:
estaciones.provincia = estaciones.provincia.map(PROV_MAP)
estaciones = estaciones.dropna()
estaciones

,id,nombre,localidad,provincia,latitud,longitud,id interno,estacion_nombre,estacion_id_interno,estacion_id
0,413,25 DE MAYO - EEA PERGAMINO,25 DE MAYO,BUENOS AIRES,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
1,413,25 DE MAYO - EEA PERGAMINO,25 DE MAYO,BUENOS AIRES,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
2,413,25 DE MAYO - EEA PERGAMINO,25 DE MAYO,BUENOS AIRES,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
3,413,25 DE MAYO - EEA PERGAMINO,25 DE MAYO,BUENOS AIRES,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
4,413,25 DE MAYO - EEA PERGAMINO,25 DE MAYO,BUENOS AIRES,-35.48,-60.13,A872872,25 de Mayo - EEA Pergamino,A872872,413
...,...,...,...,...,...,...,...,...,...,...
525,694,VILLA PARANACITO-EEA C. DEL UR,VILLA PARANACITO,ENTRE RÍOS,-33.71,-58.65,A872804,Villa Paranacito-EEA C. del Ur,A872804,694
526,694,VILLA PARANACITO-EEA C. DEL UR,VILLA PARANACITO,ENTRE RÍOS,-33.71,-58.65,A872804,Villa Paranacito-EEA C. del Ur,A872804,694
527,695,VILLA RAMALLO - EEA SAN PEDRO,VILLA RAMALLO,BUENOS AIRES,-33.52,-60.11,A872948,Villa Ramallo - EEA San Pedro,A872948,695
528,695,VILLA RAMALLO - EEA SAN PEDRO,VILLA RAMALLO,BUENOS AIRES,-33.52,-60.11,A872948,Villa Ramallo - EEA San Pedro,A872948,695


In [115]:
# joins con otras tablas

estaciones = estaciones = estaciones.merge(
    df_provincias,
    on='provincia'
).merge(
    df_localidades[['id_localidad', 'localidad']], 
    on='localidad'
)

estaciones.sample(2)

,id,nombre,localidad,provincia,latitud,longitud,id interno,estacion_nombre,estacion_id_interno,estacion_id,id_provincia,id_localidad
429,456,SANTA ROSA - EEA JUNIN MENDOZA,SANTA ROSA,MENDOZA,-33.18,-67.87,A872937,Santa Rosa - EEA Junin Mendoza,A872937,456,12,93
326,372,MANFREDI - EEA MANFREDI,MANFREDI,CÓRDOBA,-31.86,-63.75,A872907,Manfredi - EEA Manfredi,A872907,372,6,64


In [116]:
estaciones = estaciones\
    .drop(['estacion_nombre', 'localidad', 'provincia', 'estacion_id', 'id interno'], axis=1)\
    .rename(columns={'id': 'id_estacion', 'estacion_id_interno': 'id_interno'})\
    .merge(departamentos[['id_localidad', 'id_departamento']], on='id_localidad')\
    .drop_duplicates(subset='id_interno')\
    .sort_values(by='nombre')\
    .reset_index(drop=True)\
    [['id_estacion', 'id_interno', 'nombre', 'latitud', 'longitud', 'id_provincia', 'id_localidad', 'id_departamento']]

estaciones

,id_estacion,id_interno,nombre,latitud,longitud,id_provincia,id_localidad,id_departamento
0,413,A872872,25 DE MAYO - EEA PERGAMINO,-35.48,-60.13,0,0,6854
1,516,A872923,ABRA PAMPA - EEA ABRA PAMPA,-22.80,-65.83,9,78,38112
2,236,A872814,ALFONSO - EEA PERGAMINO,-33.91,-60.84,0,14,6623
3,393,A872885,ANDALGALA - EEA CATAMARCA,-27.61,-66.33,2,30,46119
4,403,A872912,ANDRESITO - EEA MONTECARLO,-25.62,-54.07,13,95,54049
...,...,...,...,...,...,...,...,...
146,511,A872963,VILLA ANA - EEA RECONQUISTA,-28.44,-59.80,20,134,82049
147,477,A872826,VILLA DOLORES - EEA MANFREDI,-31.94,-65.22,6,64,46091
148,420,A872930,VILLA MERCEDES - EEA SAN LUIS,-33.66,-65.41,18,115,74035
149,694,A872804,VILLA PARANACITO-EEA C. DEL UR,-33.71,-58.65,7,75,30056


In [117]:
crear_tabla('estaciones', estaciones)

2025-10-24 20:22:45,358 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-24 20:22:45,360 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("estaciones")
2025-10-24 20:22:45,361 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:45,362 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("estaciones")
2025-10-24 20:22:45,363 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 20:22:45,365 INFO sqlalchemy.engine.Engine 
CREATE TABLE estaciones (
	id_estacion BIGINT, 
	id_interno TEXT, 
	nombre TEXT, 
	latitud FLOAT, 
	longitud FLOAT, 
	id_provincia BIGINT, 
	id_localidad BIGINT, 
	id_departamento TEXT
)


2025-10-24 20:22:45,365 INFO sqlalchemy.engine.Engine [no key 0.00057s] ()
2025-10-24 20:22:45,370 INFO sqlalchemy.engine.Engine INSERT INTO estaciones (id_estacion, id_interno, nombre, latitud, longitud, id_provincia, id_localidad, id_departamento) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2025-10-24 20:22:45,371 INFO sqlalchemy.engine.Engine [generated in 0.00109s] [(413, 'A

## Chequeo de los datos en *`database.db`*

In [118]:
conn = sqlite3.connect('../database.db')
cursor = conn.cursor()

In [119]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('clima',),
 ('contagios',),
 ('calendario',),
 ('provincias',),
 ('localidades',),
 ('departamentos',),
 ('estaciones',)]